# Amazon Deep Learning Containers (DLC) - Extending base containers
### Installing libraries required for stable diffusion-2.1

1. [Introduction](#Introduction)  
2. [Development Environment and Permissions](#Development-Environment-and-Permissions)
3. [ECR Login](#ECR-Login)
4. [Dockerfile](#Dockerfile)
5. [Image Creation and Push to ECR](#image-creation-and-push-to-ecr)
6. [Manual image push](#manual-image-push)



# Introduction

Welcome to our extending AWS deep learning container example. In this notebook we will show you how to extend a base DLC image with libraries that fits your specific needs. We will begin by specifying a base image that we will retrieve from Amazon Elastic Container Registry (ECR). Then, we will proceed to install additional Python libraries necessary for running stable diffusion-2.1.

*To get started you'll need an aws account to access ECR.*

# Development Environment and Permissions

## Permissions
If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker.



In [ ]:
import sagemaker
import boto3
import os
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

# Check if the "container" directory exists, and create it if it doesn't
container_dir = "container"
if not os.path.exists(container_dir):
    os.makedirs(container_dir)

## ECR Login

In order to pull base images from ECR, it is necessary to authenticate with AWS. Please specify the region where you intend to use this.

In [ ]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

## Dockerfile

We are using a Hugging Face PyTorch inference ECR image as the base and extending it to include the required Python libraries for running stable diffusion.

In [4]:
%%writefile ./container/Dockerfile

ARG REGION=us-west-2

# Use the SageMaker PyTorch image as the base
FROM 763104351884.dkr.ecr.${REGION}.amazonaws.com/huggingface-pytorch-inference:1.10.2-transformers4.17.0-gpu-py38-cu113-ubuntu20.04

# Install necessary Python libraries
RUN pip install deepspeed diffusers==0.11.1 transformers==4.25.1 scipy==1.9.3 accelerate==0.15.0

Writing ./container/Dockerfile


## Image Creation and Push to ECR

Before proceeding with this step, please ensure that your execution role has the necessary access permissions for ECR. Additionally, if you encounter any issues, please verify whether the ECR repository has been created. It is important to confirm these prerequisites before continuing with image creation and pushing to ECR.

In [14]:
%%sh

# The name of our algorithm
algorithm_name=pytorch-extending-deepspeed-stable-diffusion-v2-1-base

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Enter here your aws Account ID
account_id=12345678

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the SageMaker PyTorch image
$(aws ecr get-login --registry-ids ${account_id} --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}


Process is interrupted.


## Manual Image Push

In case you encounter any issues during the previous step, you have the option to manually push the image to ECR again. This allows you to retry the image push process if needed without rebuilding your image.

In [ ]:
# Enter your own AWS Account ID
!docker push 12345678.dkr.ecr.us-west-2.amazonaws.com/pytorch-extending-deepspeed-stable-diffusion-v2-1-base